<a href="https://colab.research.google.com/github/puzzy8338/chatgpt-proxy/blob/main/00981A_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
from google.colab import files

# ✅ 使用者輸入兩個日期（格式：YYYYMMDD）
date_today = input("請輸入『今日』的日期（格式：YYYYMMDD）：").strip()
date_old = input("請輸入『比較日』的日期（格式：YYYYMMDD）：").strip()

file_today = f"ETF_Investment_Portfolio_{date_today}.xlsx"
file_old = f"ETF_Investment_Portfolio_{date_old}.xlsx"

# ✅ 檢查檔案是否已存在
missing_files = [f for f in [file_today, file_old] if not os.path.exists(f)]
if missing_files:
    print("🔍 無法在目前目錄中找到以下檔案，請上傳：", missing_files)
    uploaded = files.upload()
    for f in missing_files:
        if f not in uploaded:
            raise Exception(f"❌ 未上傳必要檔案：{f}")

# ✅ 載入原始檔案（不設欄位）
df_today_raw = pd.read_excel(file_today, header=None)
df_old_raw = pd.read_excel(file_old, header=None)

# ✅ 自動偵測表頭列
def find_stock_header_index(df_raw):
    keywords = ['股票', '股票代號', '股票名稱', '股數']
    for idx, row in df_raw.iterrows():
        cells = row.astype(str).tolist()
        if all(any(k in c for c in cells) for k in keywords):
            return idx
    return None

header_idx_today = find_stock_header_index(df_today_raw)
header_idx_old = find_stock_header_index(df_old_raw)

if header_idx_today is None or header_idx_old is None:
    raise Exception("❌ 找不到包含「股票代號」「股票名稱」「股數」等欄位的表格，請檢查 Excel 格式")

# ✅ 重新讀取並處理必要欄位
df_today = pd.read_excel(file_today, header=header_idx_today)
df_old = pd.read_excel(file_old, header=header_idx_old)
required_cols = ['股票代號', '股票名稱', '股數']

if not all(col in df_today.columns for col in required_cols) or not all(col in df_old.columns for col in required_cols):
    raise Exception("❌ 缺少必要欄位：股票代號、股票名稱、股數")

# ✅ 清理空格、欄位重新命名
for df in [df_today, df_old]:
    df['股票代號'] = df['股票代號'].astype(str).str.strip()
    df['股票名稱'] = df['股票名稱'].astype(str).str.strip()

df_today = df_today[required_cols].copy()
df_today.rename(columns={'股數': '股數_new', '股票名稱': '股票名稱_new'}, inplace=True)

df_old = df_old[required_cols].copy()
df_old.rename(columns={'股數': '股數_old', '股票名稱': '股票名稱_old'}, inplace=True)

# ✅ 合併、轉換股數、計算變化
merged = pd.merge(df_old, df_today, on='股票代號', how='outer')
merged['股票名稱'] = merged['股票名稱_new'].combine_first(merged['股票名稱_old'])

merged['股數_old'] = pd.to_numeric(merged['股數_old'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
merged['股數_new'] = pd.to_numeric(merged['股數_new'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
merged['股數變化'] = merged['股數_new'] - merged['股數_old']

# ✅ 輸出結果
result = merged[['股票代號', '股票名稱', '股數_old', '股數_new', '股數變化']].copy()
result.columns = ['股票代號', '股票名稱', '前股數', '今股數', '股數變化']

output_file = f"stock_changes_{date_old}_to_{date_today}.xlsx"
result.to_excel(output_file, index=False)
print("✅ 報告已產生，請點擊下方下載：")
files.download(output_file)


請輸入『今日』的日期（格式：YYYYMMDD）：20251021
請輸入『比較日』的日期（格式：YYYYMMDD）：20251020
🔍 無法在目前目錄中找到以下檔案，請上傳： ['ETF_Investment_Portfolio_20251021.xlsx', 'ETF_Investment_Portfolio_20251020.xlsx']


Saving ETF_Investment_Portfolio_20251021.xlsx to ETF_Investment_Portfolio_20251021.xlsx
Saving ETF_Investment_Portfolio_20251020.xlsx to ETF_Investment_Portfolio_20251020.xlsx


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


✅ 報告已產生，請點擊下方下載：


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')